<a href="https://colab.research.google.com/github/yunjeong-chang/My_Data_project/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98/%ED%87%B4%EA%B7%BC%EC%8B%9C%EA%B0%84%20%EB%B2%84%EC%8A%A4%20%EC%8A%B9%EC%B0%A8%EC%9D%B8%EC%9B%90%20%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8%20%EA%B5%AC%EC%B6%95%EA%B3%BC%20%EA%B2%80%EC%A6%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/3장-퇴근시간 버스 승차인원 예측/df_data.csv')
df.head(3)

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,cue,weekdaymean,in_out_mean,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum,bus_interval,school,transfer,holiday,68a,810a,1012a,68b,810b,1012b,ride_sum,...,teen,elder,adult_prop,kids_prop,teen_prop,elder_prop,dis_jejusi,dis_seoquiposi,dis_jejusicheong1,dis_jejuairport,dis_hallahosp,dis_rotary,dis_jejucenterhigh,dis_jejumarket,dis_jejusclass,dis_centerroad,dis_fiveway,dis_law,69a,912a,69b,912b,1820_r_mean,1820_r_sum,1820_s_mean,1820_s_sum,1820_w_mean,1820_w_sum,1820_rs_mean,1820_rs_sum,bus_route_id2,station_code2,weekend,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,2019-09-01,4270000,1,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,34.0,0,0,0,1.0,7.0,8.0,0.0,0.0,0.0,16.0,...,38.0,0.0,0.986696,0.008708,0.004596,0.0,2.954920,26.256744,3.547051,1.853545,0.719000,0.330360,4.226827,4.030090,1.897319,3.632769,1.243233,3.835351,3.0,13.0,0.0,0.0,3.104381,2409.0,1.466667,44.0,1.034282,54306.0,1.466667,44.0,625,322,1,0,0,0,0,0,0,1
1,1,2019-09-01,4270000,1,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,34.0,0,0,0,5.0,6.0,11.0,0.0,0.0,0.0,22.0,...,38.0,0.0,0.986696,0.008708,0.004596,0.0,3.720275,26.403025,4.336294,2.057088,0.088058,1.109489,5.029806,4.704476,1.501101,4.311404,0.493820,4.640172,9.0,13.0,0.0,0.0,3.104381,2409.0,4.178218,6330.0,1.034282,54306.0,5.366667,161.0,625,335,1,0,0,0,0,0,0,1
2,2,2019-09-01,4270000,1,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,34.0,0,0,0,2.0,2.0,0.0,0.0,0.0,0.0,4.0,...,38.0,0.0,0.986696,0.008708,0.004596,0.0,5.036124,25.893305,5.611975,3.316057,1.448297,2.410695,6.156335,6.069228,0.957447,5.675079,0.873821,5.840420,2.0,2.0,0.0,0.0,3.104381,2409.0,2.169559,1625.0,1.034282,54306.0,1.600000,48.0,625,408,1,0,0,0,0,0,0,1


In [ ]:
# 문자 형태의 변수 추출
object_column = list(df.columns[df.dtypes == 'object'])
# 문자형 변수 제거
df = df.drop(object_column, axis='columns')

# datetime 형태의 변수 추출
datetime_column = list(df.columns[df.dtypes == 'datetime64[ns]'])
# datetime형 변수 제거
df = df.drop(datetime_column, axis='columns')

print(object_column + datetime_column)

['date', 'station_name', 'route_station']


In [ ]:
df.shape

(643593, 83)

In [ ]:
# 학습 데이터
x_train = df.query("cue=='0'").drop(['cue', '18~20_ride'], axis='columns')
# 타겟 변수
y_train = df.query("cue=='0'")['18~20_ride']
# 테스트 데이터
x_test = df.query("cue=='1'").drop(['cue', '18~20_ride'], axis='columns')

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(415423, 81)
(415423,)
(228170, 81)


In [ ]:
x_train.head(3)

,id,bus_route_id,in_out,station_code,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,weekdaymean,in_out_mean,6~7_ride_sum,7~8_ride_sum,8~9_ride_sum,9~10_ride_sum,10~11_ride_sum,bus_interval,school,transfer,holiday,68a,810a,1012a,68b,810b,1012b,ride_sum,takeoff_sum,1012a_mean,1012a_sum,adult,...,teen,elder,adult_prop,kids_prop,teen_prop,elder_prop,dis_jejusi,dis_seoquiposi,dis_jejusicheong1,dis_jejuairport,dis_hallahosp,dis_rotary,dis_jejucenterhigh,dis_jejumarket,dis_jejusclass,dis_centerroad,dis_fiveway,dis_law,69a,912a,69b,912b,1820_r_mean,1820_r_sum,1820_s_mean,1820_s_sum,1820_w_mean,1820_w_sum,1820_rs_mean,1820_rs_sum,bus_route_id2,station_code2,weekend,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
0,0,4270000,1,344,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,34.0,0,0,0,1.0,7.0,8.0,0.0,0.0,0.0,16.0,0.0,5.304348,244.0,8158.0,...,38.0,0.0,0.986696,0.008708,0.004596,0.0,2.954920,26.256744,3.547051,1.853545,0.719000,0.330360,4.226827,4.030090,1.897319,3.632769,1.243233,3.835351,3.0,13.0,0.0,0.0,3.104381,2409.0,1.466667,44.0,1.034282,54306.0,1.466667,44.0,625,322,1,0,0,0,0,0,0,1
1,1,4270000,1,357,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,34.0,0,0,0,5.0,6.0,11.0,0.0,0.0,0.0,22.0,0.0,11.608696,534.0,8158.0,...,38.0,0.0,0.986696,0.008708,0.004596,0.0,3.720275,26.403025,4.336294,2.057088,0.088058,1.109489,5.029806,4.704476,1.501101,4.311404,0.493820,4.640172,9.0,13.0,0.0,0.0,3.104381,2409.0,4.178218,6330.0,1.034282,54306.0,5.366667,161.0,625,335,1,0,0,0,0,0,0,1
2,2,4270000,1,432,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.034282,2.044345,2942.0,4518.0,5682.0,6275.0,6774.0,34.0,0,0,0,2.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,2.304348,106.0,8158.0,...,38.0,0.0,0.986696,0.008708,0.004596,0.0,5.036124,25.893305,5.611975,3.316057,1.448297,2.410695,6.156335,6.069228,0.957447,5.675079,0.873821,5.840420,2.0,2.0,0.0,0.0,3.104381,2409.0,2.169559,1625.0,1.034282,54306.0,1.600000,48.0,625,408,1,0,0,0,0,0,0,1


### **1. 모델 설정**

**모델은 랜덤 포레스트!**

### **2. 변수 선택**

**86개 -> 68개**

In [ ]:
input_var=['in_out', 'latitude', 'longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride', 
           '6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 
           'dis_jejusi', 'dis_jejusicheong1', 'dis_jejuairport', 'dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',       
           'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway', 'dis_law', 
           'weekend', 'holiday', 'ride_sum', 'takeoff_sum', '1820_r_mean', '1820_s_mean', '1820_w_mean', '1820_rs_mean',
           'station_code2', 'bus_route_id2', '6~7_ride_sum', '7~8_ride_sum', '8~9_ride_sum', '9~10_ride_sum', '10~11_ride_sum', 
           'in_out_mean', 'weekdaymean', 'adult', 'kids', 'teen', 'elder', 'adult_prop', 'kids_prop', 'teen_prop', 'elder_prop',
           '68a', '810a', '1012a', '68b', '810b', '1012b', '69a', '912a', '69b', '912b']

### **3. 하이퍼파라미터 튜닝**

**그리드 서치 vs 랜덤 서치**

In [ ]:
#랜덤 서치
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=1217)

# 탐색할 하이퍼파라미터 구간 지정
params = {'n_estimators': stats.randint(200, 500),'max_features': stats.randint(4, 8),'min_samples_leaf': stats.randint(1, 5)}
         # n_estimators : 결정 트리의 개수, max_features : 최적의 분할을 위해 고려할 피처수, min_samples_leaf : 리프노드가 되기 위해 필요한 최소한의 샘플 데이터수(과적합 제어 용도)
         
# RandomizedSearchCV를 이용해 최적의 하이퍼파라미터 탐색
rand_cv = RandomizedSearchCV(rf, param_distributions = params, n_iter=10, cv = 3, random_state = 1217, n_jobs=-1)
rand_cv.fit(x_train[input_var], y_train)

# 최적의 하이퍼파라미터 값과 성능 출력
print('최적 하이퍼파라미터: ', rand_cv.best_params_)
print('최적 하이퍼파라미터의 성능(RMSE): {:.4f}'.format(np.sqrt(rand_cv.best_score_)))

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


최적 하이퍼파라미터:  {'max_features': 7, 'min_samples_leaf': 2, 'n_estimators': 400}
최적 하이퍼파라미터의 성능(RMSE): 0.8720


### **4. 최종 모델 구축**

In [ ]:
# submission_sample 데이터 로드
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/3장-퇴근시간 버스 승차인원 예측/submission_sample.csv')

In [ ]:
# 찾은 하이퍼파라미터 적용한 랜덤 포레스트 모델 정의
model = RandomForestRegressor(random_state=1217, max_features=7, min_samples_leaf=2, n_estimators=400, n_jobs=-1)
# 모델 학습
model.fit(x_train[input_var], y_train)

# x_train 데이터에 대한 예측값 생성
pred = model.predict(x_train[input_var])
# 성능 확인
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_train, pred))

1.2283341782783914

In [ ]:
# x_test 데이터에 대한 예측값 생성
y_pred = model.predict(x_test[input_var])
# 예측값 저장
submit['18~20_ride'] = y_pred
# 제출 파일 생성
submit.to_csv('dacon_submission.csv', index = False)